In [2]:
%%writefile lambda_function.py
import json
import boto3

def lambda_handler(event, context):
    # Initialize SageMaker runtime client
    sagemaker_client = boto3.client('sagemaker-runtime')

    # SageMaker endpoint name
    endpoint_name = 'huggingface-distilbert-endpoint'

    # Get input data from event
    try:
        input_data = event.get('inputs')
        if not input_data:
            raise ValueError("Missing 'inputs' in event")
        payload = {'inputs': input_data}
    except Exception as e:
        return {
            'statusCode': 400,
            'body': json.dumps(f'Invalid input: {str(e)}')
        }

    # Invoke SageMaker endpoint
    try:
        response = sagemaker_client.invoke_endpoint(
            EndpointName=endpoint_name,
            ContentType='application/json',
            Body=json.dumps(payload)
        )
        result = json.loads(response['Body'].read().decode())
        return {
            'statusCode': 200,
            'body': json.dumps(result)
        }
    except Exception as e:
        return {
            'statusCode': 500,
            'body': json.dumps(f'Endpoint error: {str(e)}')
        }

Writing lambda_function.py


In [3]:
import zipfile
with zipfile.ZipFile('lambda_function.zip', 'w', zipfile.ZIP_DEFLATED) as z:
    z.write('lambda_function.py')

In [8]:
import boto3

lambda_client = boto3.client('lambda')
#role_arn = 'arn:aws:iam::311410995876:role/SageMaker-MLengineer'
role_arn = 'arn:aws:iam::311410995876:role/LambdaSageMakerInvokeRole'  # Use new role

with open('lambda_function.zip', 'rb') as f:
    zipped_code = f.read()

response = lambda_client.create_function(
    FunctionName='InvokeSageMakerEndpoint',
    Runtime='python3.10',
    Role=role_arn,
    Handler='lambda_function.lambda_handler',
    Code={'ZipFile': zipped_code},
    Timeout=30,
    MemorySize=512,
    Publish=True
)
print(response)

{'ResponseMetadata': {'RequestId': '070c775e-a8de-4598-9e44-3bfaa2050831', 'HTTPStatusCode': 201, 'HTTPHeaders': {'date': 'Fri, 16 May 2025 13:38:24 GMT', 'content-type': 'application/json', 'content-length': '1362', 'connection': 'keep-alive', 'x-amzn-requestid': '070c775e-a8de-4598-9e44-3bfaa2050831'}, 'RetryAttempts': 0}, 'FunctionName': 'InvokeSageMakerEndpoint', 'FunctionArn': 'arn:aws:lambda:eu-north-1:311410995876:function:InvokeSageMakerEndpoint', 'Runtime': 'python3.10', 'Role': 'arn:aws:iam::311410995876:role/LambdaSageMakerInvokeRole', 'Handler': 'lambda_function.lambda_handler', 'CodeSize': 596, 'Description': '', 'Timeout': 30, 'MemorySize': 512, 'LastModified': '2025-05-16T13:38:24.080+0000', 'CodeSha256': 'rlTwryjjVlCQ6LLwG/+UPM1kzbJ82jzHRCld9pFcZEQ=', 'Version': '1', 'TracingConfig': {'Mode': 'PassThrough'}, 'RevisionId': '2c0607a9-2507-4da2-ba8b-26582dbfe504', 'State': 'Pending', 'StateReason': 'The function is being created.', 'StateReasonCode': 'Creating', 'PackageTy

In [9]:
import json
# Test invocation
response = lambda_client.invoke(
    FunctionName='InvokeSageMakerEndpoint',
    Payload=json.dumps({'inputs': 'This product is amazing!'})
)
result = json.loads(response['Payload'].read().decode())
print(result)

{'statusCode': 200, 'body': '[{"label": "LABEL_0", "score": 0.5288344621658325}]'}


In [ ]:
import boto3
import json
import datetime

# Initialize IAM client
iam_client = boto3.client('iam')

# Function to get all roles and their details
def export_iam_roles():
    roles_data = []
    
    # List all roles
    response = iam_client.list_roles()
    roles = response['Roles']
    
    # Handle pagination
    while 'Marker' in response:
        response = iam_client.list_roles(Marker=response['Marker'])
        roles.extend(response['Roles'])
    
    for role in roles:
        role_name = role['RoleName']
        role_arn = role['Arn']
        
        # Get role details
        role_details = iam_client.get_role(RoleName=role_name)['Role']
        
        # Get trust policy
        trust_policy = role_details.get('AssumeRolePolicyDocument', {})
        
        # Get attached managed policies
        attached_policies = []
        attached_response = iam_client.list_attached_role_policies(RoleName=role_name)
        for policy in attached_response.get('AttachedPolicies', []):
            attached_policies.append({
                'PolicyName': policy['PolicyName'],
                'PolicyArn': policy['PolicyArn']
            })
        
        # Get inline policies
        inline_policies = []
        inline_response = iam_client.list_role_policies(RoleName=role_name)
        for policy_name in inline_response.get('PolicyNames', []):
            policy_doc = iam_client.get_role_policy(
                RoleName=role_name,
                PolicyName=policy_name
            )['PolicyDocument']
            inline_policies.append({
                'PolicyName': policy_name,
                'PolicyDocument': policy_doc
            })
        
        # Get permission boundary (if any)
        permission_boundary = role_details.get('PermissionsBoundary', {}).get('PermissionsBoundaryArn', None)
        
        # Structure role data
        role_data = {
            'RoleName': role_name,
            'RoleArn': role_arn,
            'TrustPolicy': trust_policy,
            'AttachedPolicies': attached_policies,
            'InlinePolicies': inline_policies,
            'PermissionBoundary': permission_boundary,
            'CreatedDate': role_details['CreateDate'].isoformat(),
            'RoleId': role_details['RoleId']
        }
        roles_data.append(role_data)
    
    return roles_data

# Export roles to JSON file
roles_data = export_iam_roles()
timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
output_file = f'iam_roles_{timestamp}.json'

with open(output_file, 'w') as f:
    json.dump(roles_data, f, indent=2)

print(f"Exported IAM roles to {output_file}")